In [1]:
import collections
import re
from d2l import torch as d2l

In [4]:
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt', '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# total lines of the text: {len(lines)}')
print(lines[0])
print(lines[10])

# total lines of the text: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [5]:
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]
    elif token == 'char':
        return [list(line) for line in lines]
    else:
        print('ERROR: unknown token type: ' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])  # 打印前11行

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [6]:
class Vocab:
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # 初始化
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        counter = count_corpus(tokens)
        # 这里lambda是匿名函数关键字
        # 其形式为 lambda 参数: 表达式
        # 对于x，x[0]是token，x[1]是频次,所以这里是按频次排序
        # sorted()默认按照升序排列，所以需要reverse=True来降序排列
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        # 设置索引到词元的映射
        # 注意，这里设置未知词元<unk>的索引为0
        self.idx_to_token = ['<unk>'] + reserved_tokens

        # 设置镜像映射
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        
        for token, freq in self._token_freqs:
            if freq < min_freq: # 排除低频词元
                break
            if token not in self.token_to_idx: # 找到了新词元
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    def unk(self):  # 未知词元的索引为0
        return 0
    
    @property
    def token_freqs(self):
        return self._token_freqs

def count_corpus(tokens):
    # 统计词元的频次
    if len(tokens) == 0 or isinstance(tokens[0], list):
        #  展平二维列表
        # 外层 for line in tokens 是遍历每一行
        # 内层 for token in line 是遍历每一行中的每一个词元
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)   

In [7]:
voc = Vocab(tokens)
print(list(voc.token_to_idx.items())[:10])  # 频次最高的

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [8]:
for i in [0, 10]:
    print('words:', tokens[i])
    print('indices:', voc[tokens[i]])

words: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
words: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
indices: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [9]:
def load_corpus_time_machine(max_tokens=-1):
    lines = read_time_machine()
    tokens = tokenize(lines, 'char') # 简化操作，使用字符级别的词元
    vocab = Vocab(tokens)
    # 展平
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

corpus, vocab = load_corpus_time_machine()
print(len(corpus), len(vocab))

170580 28
